In [ ]:
import numpy as np

import sys
from pathlib import Path

from scipy.io import savemat

# Add project paths
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Import experiment management from HDMF src folder
from fastHDMF.simulation_runner import HDMFSimulationRunner
from fastHDMF.experiment_manager import ExperimentManager
from fastHDMF.utils.data_loading import load_all_sc_matrices

experiment_manager = ExperimentManager(project_root, experiment_id="Homeostatic_Grid", results_dir="/network/iss/cohen/data/Ivan/fastHDMF")
res = experiment_manager.load_experiment_results()
config = experiment_manager.current_config

2025-12-17 15:12:17,146 - hdmf_experiment_Homeostatic_Grid - INFO - Experiment 'Homeostatic_Grid' initialized
2025-12-17 15:12:17,148 - hdmf_experiment_Homeostatic_Grid - INFO - Config: Homeostatic grid for objective rate 3.44. Used to then calculate LR->DECAY coefficients. Now supports custom parameter generation functions.
2025-12-17 15:12:17,149 - hdmf_experiment_Homeostatic_Grid - INFO - Results will be saved to: /network/iss/cohen/data/Ivan/fastHDMF/Homeostatic_Grid
2025-12-17 15:12:17,173 - hdmf_experiment_Homeostatic_Grid - INFO - Loaded SC matrices for 1 patients from Averaged_SCs/aal with config threshold 0
2025-12-17 15:12:17,177 - hdmf_experiment_Homeostatic_Grid - INFO - Searching for config file: Homeostatic_Grid.yaml
2025-12-17 15:12:17,178 - hdmf_experiment_Homeostatic_Grid - INFO - Configs directory: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs
2025-12-17 15:12:17,191 - hdmf_experiment_Homeostatic_Grid - INFO - Homeostatic_Grid
2025-12-17 15:12:17,192 - hdmf_ex

INFO: Searching for config file: Homeostatic_Grid.yaml
INFO: Configs directory: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs
INFO: Homeostatic_Grid
INFO: Found config matches: [PosixPath('/network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Homeostatic_Grid.yaml')]
INFO: Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Homeostatic_Grid.yaml
Creating IPP list from SC matrices in: Averaged_SCs/aal
Found 1 IPPs from SC matrices


In [2]:
hom_grid = np.stack(res['full_results']['observables']['mean_rates'].flatten())
hom_grid = np.reshape(hom_grid, (100,110,17,90))
hom_grid = 3.44 - np.mean(hom_grid, axis=-1)
hom_grid_mean = np.mean(hom_grid, axis=2)
hom_grid_std = np.std(hom_grid, axis=2)


In [18]:
# Parameter ranges from config (fallbacks if missing)


# Check explicit Homeostatic_Grid-like config where functions are used
def _generate_parameter_values(spec: dict) -> np.ndarray:
    """Generate parameter values from spec with optional custom function"""
    if "fun" in spec:
        fun_name = spec["fun"]
        args = spec.get("args", [])
        kwargs = spec.get("kwargs", {})
        
        try:
            func = eval(fun_name)
            values = func(*args, **kwargs)
            return np.array(values, dtype=float)
        except Exception as e:
            raise ValueError(f"Error calling function '{fun_name}': {e}")
    else:
        # Default: start, end, step
        values = np.arange(spec["start"], spec["end"], spec["step"], dtype=float)
        return np.round(values, 12)
    
over_cfg = config.get('simulation', {}).get('over', {})
G_range = _generate_parameter_values(config['simulation']['over']['G'])
LR_range = _generate_parameter_values(config['grid']['lrj'])
DECAY_range = _generate_parameter_values(config['grid']['taoj'])

nlr = len(LR_range)
ndec = len(DECAY_range)
nG = len(G_range)

# find g axis by size match
axes_sizes = hom_grid.shape
print("Assuming G Axis is the last axis")
hom_grid_mean = np.mean(hom_grid, axis=-1)
std_hom_grid = np.std(hom_grid, axis=-1)

min_mm_pos = np.argmin(np.abs(hom_grid_mean), axis=1)

x_idx = np.arange(nlr)
# Extract the decay values corresponding to the minimum mismatch for each learning rate
y = DECAY_range[min_mm_pos]
if config.get('simulation', {}).get('obj_rate', 3.44) == 1.22:
    coeff = np.polyfit(np.log(LR_range[21:]), np.log(y[21:]), 1)
    #coeff_plot = np.polyfit(np.log(x_idx[21:]), np.log(y[21:]), 1)
else:
    coeff = np.polyfit(np.log(LR_range), np.log(y), 1)
    #coeff_plot = np.polyfit(np.log(x_idx), np.log(y), 1)

print("Fit coefficients (LR, DECAY):", coeff)
np.save(project_root / Path("data") / f"fit_res_{str(config['simulation']['obj_rate']).replace('.','-')}.npy", coeff)
savemat(project_root / Path("data") / f"fit_res_{str(config['simulation']['obj_rate']).replace('.','-')}.mat", {'fit_res': coeff})


Assuming G Axis is the last axis
Fit coefficients (LR, DECAY): [-0.92416023 12.42424635]


## Save FIC for Mixed model

In [ ]:
experiment_manager = ExperimentManager(project_root, experiment_id="Dynamic_Stability", results_dir="/network/iss/cohen/data/Ivan/fastHDMF")
res_dynamic = experiment_manager.load_experiment_results()
mean_fic_dynamic = res_dynamic['full_results']['observables']['mean_fic']


2025-12-17 15:33:24,910 - hdmf_experiment_Dynamic_Stability - INFO - Experiment 'Dynamic_Stability' initialized
2025-12-17 15:33:24,912 - hdmf_experiment_Dynamic_Stability - INFO - Config: Showing firing rate stability across G values in the dynamic model
2025-12-17 15:33:24,913 - hdmf_experiment_Dynamic_Stability - INFO - Results will be saved to: /network/iss/cohen/data/Ivan/fastHDMF/Dynamic_Stability


2025-12-17 15:33:24,937 - hdmf_experiment_Dynamic_Stability - INFO - Loaded SC matrices for 1 patients from Averaged_SCs/aal with config threshold 0
2025-12-17 15:33:24,941 - hdmf_experiment_Dynamic_Stability - INFO - Searching for config file: Dynamic_Stability.yaml
2025-12-17 15:33:24,942 - hdmf_experiment_Dynamic_Stability - INFO - Configs directory: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs
2025-12-17 15:33:24,953 - hdmf_experiment_Dynamic_Stability - INFO - Dynamic_Stability
2025-12-17 15:33:24,955 - hdmf_experiment_Dynamic_Stability - INFO - Found config matches: [PosixPath('/network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Dynamic_Stability.yaml'), PosixPath('/network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/old/Dynamic_Stability.yaml')]
2025-12-17 15:33:24,973 - hdmf_experiment_Dynamic_Stability - INFO - Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Dynamic_Stability.yaml


INFO: Searching for config file: Dynamic_Stability.yaml
INFO: Configs directory: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs
INFO: Dynamic_Stability
INFO: Found config matches: [PosixPath('/network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Dynamic_Stability.yaml'), PosixPath('/network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/old/Dynamic_Stability.yaml')]
INFO: Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Dynamic_Stability.yaml
Creating IPP list from SC matrices in: Averaged_SCs/aal
Found 1 IPPs from SC matrices


In [18]:

g_values = res_dynamic['full_results']['axis_values']['G']
for idx, g in enumerate(g_values): 
    avg_fic = np.mean(mean_fic_dynamic[idx, :, :], axis=0) # Average seeds
    g_value_str = f"{g:.2f}".replace('.','')
    fname = f"mean_fic_G_{g_value_str}.npy"
    np.save(project_root / "data" / "dyn_fics" / fname, avg_fic)

In [19]:
avg_fic.shape


(90,)

In [20]:
experiment_manager = ExperimentManager(project_root, experiment_id="Dynamic_Neuromod", results_dir="/network/iss/cohen/data/Ivan/fastHDMF")
res_dynamic = experiment_manager.load_experiment_results()
mean_fic_dynamic = res_dynamic['full_results']['observables']['mean_fic']


2025-12-17 17:19:33,209 - hdmf_experiment_Dynamic_Neuromod - INFO - Experiment 'Dynamic_Neuromod' initialized
2025-12-17 17:19:33,211 - hdmf_experiment_Dynamic_Neuromod - INFO - Config: Grid of 100 neuromodulatory values
2025-12-17 17:19:33,212 - hdmf_experiment_Dynamic_Neuromod - INFO - Results will be saved to: /network/iss/cohen/data/Ivan/fastHDMF/AAL_HDMF_PAPER/Dynamic_Neuromod
2025-12-17 17:19:33,240 - hdmf_experiment_Dynamic_Neuromod - INFO - Loaded SC matrices for 1 patients from Averaged_SCs/aal with config threshold 0
2025-12-17 17:19:33,244 - hdmf_experiment_Dynamic_Neuromod - INFO - Searching for config file: Dynamic_Neuromod.yaml
2025-12-17 17:19:33,245 - hdmf_experiment_Dynamic_Neuromod - INFO - Configs directory: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs
2025-12-17 17:19:33,257 - hdmf_experiment_Dynamic_Neuromod - INFO - Dynamic_Neuromod
2025-12-17 17:19:33,259 - hdmf_experiment_Dynamic_Neuromod - INFO - Found config matches: [PosixPath('/network/iss/home/ivan

INFO: Searching for config file: Dynamic_Neuromod.yaml
INFO: Configs directory: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs
INFO: Dynamic_Neuromod
INFO: Found config matches: [PosixPath('/network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Dynamic_Neuromod.yaml'), PosixPath('/network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/old/Dynamic_Neuromod.yaml')]
INFO: Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Dynamic_Neuromod.yaml
Creating IPP list from SC matrices in: Averaged_SCs/aal
Found 1 IPPs from SC matrices


In [21]:
w_values = res_dynamic['full_results']['axis_values']['wgaine']
for idx, w in enumerate(w_values): 
    avg_fic = np.mean(mean_fic_dynamic[idx,0, :, :]) # Use the values from the lowest LR
    w_value_str = f"{w:.2f}".replace('.','')
    fname = f"mean_fic_w_{w_value_str}.npy"
    print(f"Saving FIC for w={w} to {fname}")
    
    np.save(project_root / "data" / "dyn_fics" / fname, avg_fic)

Saving FIC for w=0.01 to mean_fic_w_001.npy
Saving FIC for w=0.02 to mean_fic_w_002.npy
Saving FIC for w=0.03 to mean_fic_w_003.npy
Saving FIC for w=0.04 to mean_fic_w_004.npy
Saving FIC for w=0.05 to mean_fic_w_005.npy
Saving FIC for w=0.060000000000000005 to mean_fic_w_006.npy
Saving FIC for w=0.06999999999999999 to mean_fic_w_007.npy
Saving FIC for w=0.08 to mean_fic_w_008.npy
Saving FIC for w=0.09 to mean_fic_w_009.npy
Saving FIC for w=0.09999999999999999 to mean_fic_w_010.npy
Saving FIC for w=0.11 to mean_fic_w_011.npy
Saving FIC for w=0.12 to mean_fic_w_012.npy
Saving FIC for w=0.13 to mean_fic_w_013.npy
Saving FIC for w=0.14 to mean_fic_w_014.npy
Saving FIC for w=0.15000000000000002 to mean_fic_w_015.npy
Saving FIC for w=0.16 to mean_fic_w_016.npy
Saving FIC for w=0.17 to mean_fic_w_017.npy
Saving FIC for w=0.18000000000000002 to mean_fic_w_018.npy
Saving FIC for w=0.19 to mean_fic_w_019.npy
Saving FIC for w=0.2 to mean_fic_w_020.npy
Saving FIC for w=0.21000000000000002 to mean_

In [ ]:

g_values = res_dynamic['full_results']['axis_values']['G']
for idx, g in enumerate(g_values): 
    avg_fic = np.mean(np.mean(mean_fic_dynamic[idx, :, :],axis=1))
    g_value_str = f"{g:.2f}".replace('.','')
    fname = f"mean_fic_G_{g_value_str}.npy"
    np.save(project_root / "data" / "dyn_fics" / fname, avg_fic)